# Predecir si el ejercicio es correcto
Aplicar Machine Learning para predecir si el paciente está realizando correctamente el ejercicio (clasificación binaria)

In [ ]:
# ---------- importar librerías---------
# Manipular los datos
import pandas as pd

# Gráficas
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8')
sns.set_theme(style="darkgrid")

# scikit-learn (ML en python)
## Procesar el dataset
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
## Modelos ML
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
## Evaluación de los modelos
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
## Hiperparametrizacion
from sklearn.model_selection import GridSearchCV
## Seleccion de variables
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFECV # recursive

# Class imbalance
from imblearn.under_sampling import NearMiss, EditedNearestNeighbours
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTETomek, SMOTEENN
from collections import Counter

## Seleccion de variables
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFECV # recursive
from sklearn.decomposition import PCA

# Pipeline
from imblearn.pipeline import Pipeline


# Para ignorar los Warnings
import warnings
from sklearn.exceptions import FitFailedWarning 

warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)

In [ ]:
#---------Importar los datos-----------
# Dataframe medidas calculadas por repetición
df = pd.read_csv('../csvFiles/medidasPerRepetition.csv', dtype=object)

In [ ]:
# ------Preparar el dataset-------
encoder = OrdinalEncoder(categories=[list(set(df["Position"].values))])
encoder.fit(df[["Position"]])
df["Position"] = encoder.transform(df[["Position"]])

# pasar variable obj to numeric
df = df.apply(pd.to_numeric, errors='ignore')

# cmabiar las poorly executed to incorrectly executed
df[df['CorrectLabel']==3] = 2

In [ ]:
df

## ML simple

In [ ]:
#-------Funciones--------

# Para entrenar los modelos de ML
def aplicar_modelos(X: pd.DataFrame, y: pd.DataFrame, file):
    """
    Aplica una lista de modelos de aprendizaje automático a los datos de prueba y muestra
    los informes de evaluación.

    Parámetros:
    * X: datos de entrenamiento.
    * y: target de entrenamiento.
    * file: archivo para escribir los resultados.
    """
    # creación de los modelos
    knn = KNeighborsClassifier(n_neighbors=1)  # K-Neighbors classifier
    dtree = DecisionTreeClassifier()  # Decision tree
    nb = GaussianNB()  # Naive Bayes
    svm = SVC()  # Support Vector Machine
    rf = RandomForestClassifier(n_estimators=5)  # Random Forest
    lr = LogisticRegression(max_iter=500)  # Logistic Regression

    models = [knn, dtree, nb, svm, rf, lr]

    # Iteramos sobre los modelos
    for modelo in models:
        try:
            accuracy_scores = cross_val_score(modelo, X, y, cv=5, scoring='accuracy')
            recall_scores = cross_val_score(modelo, X, y, cv=5, scoring='recall')
            file.write(f"\n\t Modelo: {modelo.__class__.__name__}\n")
            file.write("\t%0.2f accuracy with a standard deviation of %0.2f\n" % (accuracy_scores.mean(), accuracy_scores.std()))
            file.write("\t%0.2f recall with a standard deviation of %0.2f\n" % (recall_scores.mean(), recall_scores.std()))            
        except ValueError as e:
            file.write(f"Skipping {modelo.__class__.__name__} due to ERROR: {e}\n")

# Aplicar imbalance learn
def aplicar_imblearn(X, y, file):
    nm1 = NearMiss(version=1, n_neighbors=2)
    enn = EditedNearestNeighbours()
    adasyn = ADASYN(n_neighbors=2)
    smote = SMOTE(k_neighbors=2)
    smotetomek = SMOTETomek(smote=smote)
    smote_enn = SMOTEENN(smote=smote)

    imblearns = [nm1, enn, adasyn, smote, smotetomek, smote_enn]
    for imblearn in imblearns:
        try:
            X_im, y_im = imblearn.fit_resample(X, y)
            file.write(f"\n- MODELO DE RESAMPLING: {imblearn.__class__.__name__}\n")
            file.write("Distribution of class labels before resampling {}\n".format(Counter(y)))
            file.write("Distribution of class labels after resampling {}\n".format(Counter(y_im)))
            aplicar_modelos(X_im, y_im, file)
        except ValueError as e:
            file.write(f"Skipping {imblearn.__class__.__name__} due to ERROR: {e}\n")
        except RuntimeError as e:
            file.write(f"Skipping {imblearn.__class__.__name__} due to ERROR: {e}\n")

# dividir en test y train df_gestures
def split_df_gestures(df, target):
    #---------Dividimos el dataset------------
    # dividimos en variables independientes y en target
    X = df.drop([target], axis=1)
    y = df[target]

    #---------Dividimos en train y test------------
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)    

    return X_train, X_test, y_train, y_test

In [ ]:
gestures = [0, 1, 2, 4, 5, 6, 7, 8] #df['GestureLabel'].unique()

with open('cvResults.txt', 'w') as file:
    for gesture in gestures:
        df_gesture = df[df['GestureLabel'] == gesture]
        #---------Dividimos en train y test------------
        X_train, X_test, y_train, y_test = split_df_gestures(df_gesture, 'CorrectLabel')  

        file.write(f'\n------------ CLASIFICACION GESTO {gesture} -----------\n')
        aplicar_imblearn(X_train, y_train, file)

## Pipeline con hiperparametrizacion y feature selection

In [ ]:
# Para ignorar los FitFileWarning
warnings.filterwarnings("ignore", category=FitFailedWarning)

# definimos el param_grid
param_grid = [
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.8, 0.90, 1],
        'classifier': [KNeighborsClassifier(n_neighbors=2)],
        'classifier__n_neighbors': [2, 3, 5], 
        'classifier__weights': ['uniform', 'distance'], 
        'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  
    },
    {
        'reduce_dim__n_components': [0.7,0.8, 0.90, 1],
        'classifier': [DecisionTreeClassifier()],
        'classifier__criterion': ['gini', 'entropy'],
        'classifier__max_depth': [None, 10, 20, 30]
    },
    {
        'reduce_dim__n_components': [0.7, 0.8, 0.90, 1],
        'classifier': [GaussianNB()]
    },
    {
        'reduce_dim__n_components': [0.7, 0.8, 0.90, 1],
        'classifier': [SVC()],
        'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'classifier__C': [0.1, 1, 10, 100]
    },
    {

        'reduce_dim__n_components': [0.7, 0.8, 0.90, 1],
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_features': ['auto', 'sqrt', 'log2']
    },
    {

        'reduce_dim__n_components': [0.7, 0.8, 0.90, 1],
        'classifier': [LogisticRegression(max_iter=500)],
        'classifier__C': [0.01, 0.1, 1, 10, 100]
    }
]

with open('resultsPipeline.txt', 'w') as file:
    for gesture in df['GestureLabel'].unique():

        df_gesture = df[df['GestureLabel'] == gesture]

        #---------Dividimos en train y test------------
        X_train, X_test, y_train, y_test = split_df_gestures(df_gesture)   
        
        file.write(f'\n------------ CLASIFICACION GESTO {gesture} -----------\n')
        print(f'\n------------ CLASIFICACION GESTO {gesture} -----------\n')
        pipeline = Pipeline([('balance_data',  SMOTETomek(smote=SMOTE(k_neighbors=2))),
                            ('reduce_dim', PCA()),
                            ('classifier', KNeighborsClassifier(n_neighbors=2))])

        # Create the GridSearchCV object
        grid = GridSearchCV(pipeline, param_grid)
        grid.fit(X_train, y_train)
        best_pipeline = grid.best_estimator_

        file.write('Training set score: ' + str(best_pipeline.score(X_train, y_train)))
        file.write('Test set score: ' + str(best_pipeline.score(X_test, y_test)))

        print('Training set score: ' + str(best_pipeline.score(X_train, y_train)))
        print('Test set score: ' + str(best_pipeline.score(X_test, y_test)))

        globals()[f'best_pipe_{gesture}'] = best_pipeline

